In [ ]:
import gradio as gr
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from datetime import datetime
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from sentence_transformers import SentenceTransformer, util
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ✅ Load environment variables
load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")

# ✅ Load dataset and train model
df = pd.read_csv("Crop_recommendation.csv")
X = df[["temperature", "humidity"]]
y = df["label"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# ✅ NER Setup
ner_model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=tokenizer, grouped_entities=True)

# ✅ Sentence Embedding WSD
wsd_model = SentenceTransformer('all-MiniLM-L6-v2')
sense_inventory = {
    "plant_1": "a living organism like a crop or tree",
    "plant_2": "an industrial facility such as a pesticide plant",
    "spray_1": "the act of spraying liquid on crops",
    "spray_2": "a physical pesticide product in a bottle"
}

def disambiguate_word(context_sentence, word):
    senses = [k for k in sense_inventory if k.startswith(word)]
    if not senses:
        return None
    context_emb = wsd_model.encode(context_sentence, convert_to_tensor=True)
    best_sense, best_score = None, -1
    for sense_key in senses:
        gloss_emb = wsd_model.encode(sense_inventory[sense_key], convert_to_tensor=True)
        score = util.pytorch_cos_sim(context_emb, gloss_emb).item()
        if score > best_score:
            best_sense = sense_key
            best_score = score
    return best_sense

# ✅ Extract location
def extract_location(text):
    entities = ner_pipeline(text)
    for ent in entities:
        if ent["entity_group"] == "LOC":
            return ent["word"]
    return None

# ✅ Weather Fetching
def get_weather_data(location):
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={API_KEY}&units=metric"
        res = requests.get(url).json()
        if res.get("cod") != 200:
            return None, None, None
        return res['weather'][0]['description'], res['main']['temp'], res['main']['humidity']
    except:
        return None, None, None

# ✅ Main Query Handler
def handle_query(query):
    if not query.strip():
        return "❌ Please enter a valid query."

    # Disambiguation
    if "plant" in query.lower():
        sense = disambiguate_word(query, "plant")
        if sense == "plant_2":
            return "Did you mean a factory? For crops, please rephrase your question."

    # Extract location
    location = extract_location(query)
    if not location:
        return "❌ Could not detect a location. Please include a city name in your query."

    weather_desc, temp, humidity = get_weather_data(location)
    if temp is None:
        return f"⚠️ Couldn't fetch weather for {location}. Please try another location."

    # Crop or Weather intent
    crop_keywords = ["crop", "grow", "plant", "farming", "recommend"]
    weather_keywords = ["weather", "temperature", "climate", "humid", "rain", "forecast"]

    if any(word in query.lower() for word in crop_keywords):
        # Predict crop
        input_data = np.array([[temp, humidity]])
        crop_encoded = model.predict(input_data)[0]
        crop_label = label_encoder.inverse_transform([crop_encoded])[0]
        return f"📍 **City**: {location}\n🌤️ **Weather**: {weather_desc}\n🌡️ **Temperature**: {temp}°C\n💧 **Humidity**: {humidity}%\n🌱 **Recommended Crop**: {crop_label}"

    elif any(word in query.lower() for word in weather_keywords):
        return f"📍 **City**: {location}\n🌤️ **Weather**: {weather_desc}\n🌡️ **Temperature**: {temp}°C\n💧 **Humidity**: {humidity}%"

    else:
        return f"📍 **City**: {location}\n🌤️ **Weather**: {weather_desc}\n🌡️ **Temperature**: {temp}°C\n💧 **Humidity**: {humidity}%"

# ✅ Gradio Interface
demo = gr.Interface(
    fn=handle_query,
    inputs=gr.Textbox(lines=2, placeholder="Ask about crops or weather..."),
    outputs="text",
    title="🌾 Smart AgriBot",
    description="Ask about crops or weather in your city. E.g. 'What crop should I grow in Pune?' or 'Tell me the weather in Delhi.'"
)

demo.launch()


c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Device set to use cpu
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
